In [1]:
import pandas as pd
import numpy as np
import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [2]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/new_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels20.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno

In [3]:
'''
Example of simple tuning bm25 and its not saving to the folder 
'''

import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes"
# folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
folders = ['webd-dependency', 'combined-fasttext', 'essays-dependency',
          'original', 'essays-fasttext', 'ibm-fasttext', 'webd-fasttext']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            if subdir.endswith('100'):
                print(path)
                indexref = pt.IndexRef.of(path)
                index = pt.IndexFactory.of(indexref)
                bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"c" : 0.75, "bm25.k_1": 1.2, "bm25.k_3": 8})
                tuned_bm25, parameters = pt.KFoldGridSearch(
                    bm25,
                    {bm25: {"c" : [0.15, 0.35, 0.55, 0.75],
                    "bm25.k_1": [0.6, 1.2, 1.6, 2.2, 2.8, 3.4, 4.0, 4.4],
                    "bm25.k_3": [2, 5, 8, 10]
                    }},
                    [topics20, topics21],
                    [qrels20, qrels21],
                    objective
                )
                
                print(pt.Experiment([tuned_bm25], topics, qrels, names=[folders[i] + '_tuned_bm25_'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
                print(pt.Experiment([tuned_bm25], topics20, qrels, names=[folders[i] + '_tuned_bm25_20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
                print(pt.Experiment([tuned_bm25], topics21, qrels, names=[folders[i] + '_tuned_bm25_21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))

                print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm25__unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
                print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)]], topics20, qrels, names=[folders[i] + '_tuned_bm25__unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
                print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)]], topics21, qrels, names=[folders[i] + '_tuned_bm25__unjudged21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))

                i += 1
            

/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_webd-dependency-100/data.properties
Fold 1
Best ndcg_cut_5 is 0.374316
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=1.2', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.228875
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=1.6', 'BR(BM25) bm25.k_3=2']
                          name  ndcg_cut_5  ndcg_cut_25    bpref
0  webd-dependency_tuned_bm25_    0.298138     0.216233  0.34511
                            name  ndcg_cut_5  ndcg_cut_25     bpref
0  webd-dependency_tuned_bm25_20    0.225169     0.155344  0.315279
                            name  ndcg_cut_5  ndcg_cut_25     bpref
0  webd-dependency_tuned_bm25_21    0.369648     0.275904  0.374344
                                   name  ndcg_cut_5  ndcg_cut_25    bpref
0  webd-dependency_tuned_bm25__unjudged    0.582475     0.463565  0.34511
                                     name  ndcg_cut_5  ndcg_cut_25     bpre